<a href="https://colab.research.google.com/github/utkarsh599/Recomendation-sysy/blob/Abhijeet/learningSpark6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-eu.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

In [2]:
!pip install py4j

     |████████████████████████████████| 204kB 3.4MB/s 


In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

In [0]:
import findspark
findspark.init()

In [0]:
import pyspark

In [6]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
from pyspark.sql import SparkSession


In [0]:
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [0]:
comments1 =  spark.read.csv("/content/gdrive/My Drive/Colab Notebooks/GBcomments.csv", inferSchema=True, header=True)

In [0]:
comments2= spark.read.csv("/content/gdrive/My Drive/Colab Notebooks/UScomments.csv", inferSchema=True, header=True)

In [11]:
comments1.show(5)

+-----------+--------------------+-----+-------+
|   video_id|        comment_text|likes|replies|
+-----------+--------------------+-----+-------+
|jt2OHQh0HoQ|It's more accurat...|    0|      0|
|jt2OHQh0HoQ|To be there with ...|    1|      0|
|jt2OHQh0HoQ|Thank gosh, a pla...|    0|      0|
|jt2OHQh0HoQ|What happened to ...|    0|      0|
|jt2OHQh0HoQ|Power is the dise...|    0|      0|
+-----------+--------------------+-----+-------+
only showing top 5 rows



In [12]:
type(comments1)

pyspark.sql.dataframe.DataFrame

In [0]:
from functools import reduce  # For Python 3.x
from pyspark.sql import DataFrame

def unionAll(*dfs):
    return reduce(DataFrame.unionAll, dfs)

comments=unionAll(comments1,comments2)

In [14]:
comments.count()

1410466

In [15]:
comments_rdd = comments.select('comment_text').rdd.flatMap(lambda x: x)
comments_rdd=comments_rdd.filter(lambda x: x is not None).filter(lambda x: x!='')
comments_rdd.take(5)

["It's more accurate to call it the M+ (1000) because the price is closer than calling it the X (10).",
 'To be there with a samsung phone\\n😂😂😂',
 'Thank gosh, a place I can watch it without having to be at HD... my speed doesn’t support HD',
 'What happened to the home button on the iPhone X? *****Cough****copying Samsung******cough',
 'Power is the disease.\xa0 Care is the cure.\xa0 Keep caring for yourself and others as best as you can.\xa0 This is life.']

In [0]:
header = comments_rdd.first()

In [0]:
data_rmv_col = comments_rdd.filter(lambda row: row != header)

In [0]:
def lower(x):
  return x.lower()

In [0]:
lowerCase_sentRDD = data_rmv_col.map(lower)

In [20]:
lowerCase_sentRDD.take(5)

['to be there with a samsung phone\\n😂😂😂',
 'thank gosh, a place i can watch it without having to be at hd... my speed doesn’t support hd',
 'what happened to the home button on the iphone x? *****cough****copying samsung******cough',
 'power is the disease.\xa0 care is the cure.\xa0 keep caring for yourself and others as best as you can.\xa0 this is life.',
 'keep calm and buy iphone 8 keep calm and buy iphone 8 plus keep calm and buy iphone x. what is your favourite?']

In [0]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [0]:
def sent_TokenizeFunct(x):
    doc=nlp(x)
    #return doc
    tokens=[]
    for token in doc:
      if((token.is_stop==False and token.is_punct==False) and token.is_alpha==True):
        tokens.append(token.lemma_)
    return ' '.join(tokens)
    
sentenceTokenizeRDD = lowerCase_sentRDD.map(sent_TokenizeFunct)

In [23]:
sentenceTokenizeRDD.take(5)

['samsung',
 'thank gosh place watch have hd speed support hd',
 'happen home button iphone x',
 'power disease care cure care best life',
 'calm buy iphone calm buy iphone plus calm buy iphone favourite']

# Working on Videos

In [0]:
Videos1 =  spark.read.csv("/content/gdrive/My Drive/Colab Notebooks/GBvideos.csv", inferSchema=True, header=True)
Videos2 =  spark.read.csv("/content/gdrive/My Drive/Colab Notebooks/USvideos.csv", inferSchema=True, header=True)
videos = unionAll(Videos1,Videos2)

In [25]:
videos.show(5)

+-----------+--------------------+-----------------+-----------+--------------------+-------+-----+--------+-------------+--------------------+-----+
|   video_id|               title|    channel_title|category_id|                tags|  views|likes|dislikes|comment_total|      thumbnail_link| date|
+-----------+--------------------+-----------------+-----------+--------------------+-------+-----+--------+-------------+--------------------+-----+
|jt2OHQh0HoQ|Live Apple Event ...|      Apple Event|         28|apple events|appl...|7426393|78240|   13548|          705|https://i.ytimg.c...|13.09|
|AqokkXoa7uE|Holly and Phillip...|     This Morning|         24|this morning|inte...| 494203| 2651|    1309|            0|https://i.ytimg.c...|13.09|
|YPVcg45W0z4|My DNA Test Resul...|     emmablackery|         24|emmablackery|emma...| 142819|13119|     151|         1141|https://i.ytimg.c...|13.09|
|T_PuZBdT2iM|getting into a co...|            ProZD|          1|skit|korean|langu...|1580028|65729| 

In [26]:
videos_rdd = videos.select(['title']).rdd.flatMap(lambda x: x)
videos_rdd = videos_rdd.filter(lambda x: x is not None).filter(lambda x: x!='')
videos_rdd.take(5)

['Live Apple Event - Apple September Event 2017 - iPhone 8, iPhone X, iOS 11 - Apple Keynote',
 'Holly and Phillip Meet Samantha the Sex Robot | This Morning',
 "My DNA Test Results! I'm WHAT?!",
 "getting into a conversation in a language you don't actually speak that well",
 'Baby Name Challenge!']

In [0]:
header_Vid=videos_rdd.first()

In [0]:
data_rmv_col_vid = videos_rdd.filter(lambda row: row != header_Vid)

In [0]:
lowerCase_VidRDD = data_rmv_col_vid.map(lower)

In [30]:
lowerCase_VidRDD.take(5)

['holly and phillip meet samantha the sex robot | this morning',
 "my dna test results! i'm what?!",
 "getting into a conversation in a language you don't actually speak that well",
 'baby name challenge!',
 "revealed - fifa 18 stats for chelsea's hazard, luiz & christensen!"]

In [0]:
Vid_sentTokenizeRDD = lowerCase_VidRDD.map(sent_TokenizeFunct)

In [32]:
Vid_sentTokenizeRDD.take(5)

['holly phillip meet samantha sex robot morning',
 'dna test result',
 'get conversation language actually speak',
 'baby challenge',
 'reveal fifa stat chelsea hazard luiz christensen']

In [33]:
tags_rdd=videos.select(['tags']).rdd.flatMap(lambda x:x)
tags_rdd.take(5)

['apple events|apple event|iphone 8|iphone x|iphone 8 plus|iphone 7s|iphone 7s plus|ios 11',
 'this morning|interview|holly willoughby|phillip schofield|ruth langsford|eamonn holmes|chat shows - topic|chat show - topic|talk shows - topic|sex robot|samantha|real life',
 'emmablackery|emma blackery|emma|blackery|british vlogger|british youtuber|female vlogger|birdyboots|dna test|23andme|ancestry',
 "skit|korean|language|conversation|esl|japanese|foreign|communication|don't speak|struggle|foreigner|lotte mart|korea|south|weather|good|tutorial",
 'sprinkleofglitter|sprinkle of glitter|baby glitter|Louise Pentland|Louise Pentland Baby Name|Mrs Meldrum|Baby Names|Nice baby names|Unusual baby names']

In [0]:
header_tags = tags_rdd.first()

In [0]:
tags_rmv_col = tags_rdd.filter(lambda row: row != header_tags)

In [36]:
tags_rmv_col.take(5)

['this morning|interview|holly willoughby|phillip schofield|ruth langsford|eamonn holmes|chat shows - topic|chat show - topic|talk shows - topic|sex robot|samantha|real life',
 'emmablackery|emma blackery|emma|blackery|british vlogger|british youtuber|female vlogger|birdyboots|dna test|23andme|ancestry',
 "skit|korean|language|conversation|esl|japanese|foreign|communication|don't speak|struggle|foreigner|lotte mart|korea|south|weather|good|tutorial",
 'sprinkleofglitter|sprinkle of glitter|baby glitter|Louise Pentland|Louise Pentland Baby Name|Mrs Meldrum|Baby Names|Nice baby names|Unusual baby names',
 'Rio Ferdinand|Rio|Ferdinand|5 Magazine|5 Mag|#5|5 Rio Ferdinand|Rio Ferdinand interview|football|soccer|interview|Manchester United|Man United|fifa 18|fut 18|fut|fifa ratings|fifa 18 ratings|chelsea|chelsea fc|chelsea f.c|David Luiz|Eden Hazard|Christensen|Premier League|Champions League|Fifa18|Hazard Fifa 18|David Luiz Fifa 18|Christensen Fifa 18|Chelsea Fifa 18|Pack Opening|Fifa Pack

In [0]:
def split1(x):
  return (x.split("|"))

tags_rdd=tags_rmv_col.map(split1)

In [0]:
from pyspark.sql import Row
from pyspark.sql.functions import monotonically_increasing_id

row_title = Row("LemmetizeTitle") # Or some other column name
Title_DF = Vid_sentTokenizeRDD.map(row_title).toDF()
Title_DF = Title_DF.select("*").withColumn("id", monotonically_increasing_id())

row_tags=Row("Tokenized_Tags")
Tags_DF=tags_rdd.map(row_tags).toDF()
Tags_DF = Tags_DF.select("*").withColumn("id", monotonically_increasing_id())

In [39]:
Title_DF.show(5)

+--------------------+---+
|      LemmetizeTitle| id|
+--------------------+---+
|holly phillip mee...|  0|
|     dna test result|  1|
|get conversation ...|  2|
|      baby challenge|  3|
|reveal fifa stat ...|  4|
+--------------------+---+
only showing top 5 rows



In [40]:
Tags_DF.show(5)

+--------------------+---+
|      Tokenized_Tags| id|
+--------------------+---+
|[this morning, in...|  0|
|[emmablackery, em...|  1|
|[skit, korean, la...|  2|
|[sprinkleofglitte...|  3|
|[Rio Ferdinand, R...|  4|
+--------------------+---+
only showing top 5 rows



In [41]:
Tags_DF[['Tokenized_Tags']]

DataFrame[Tokenized_Tags: array<string>]

In [0]:
videos_DF = Title_DF.join(Tags_DF, on=['id'], how='left_outer')

In [43]:
videos_DF.show(5)

+----+--------------------+--------------------+
|  id|      LemmetizeTitle|      Tokenized_Tags|
+----+--------------------+--------------------+
|  26|testing soph x ma...|[grey hair, sophd...|
|  29|move vlog pink ho...|[moving vlog, ike...|
| 474|doom wolfenstein ...|[PC, IGN, PS4, Do...|
| 964|lloyd new lego ni...|[Google, YouTube,...|
|1677|project car ceo q...|[Project Cars 2 G...|
+----+--------------------+--------------------+
only showing top 5 rows



In [44]:
videos_DF.count()

15989

In [45]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer


tokenizer = Tokenizer(inputCol="LemmetizeTitle", outputCol="words")
wordsData = tokenizer.transform(videos_DF)

hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=5)
featurizedData = hashingTF.transform(wordsData)

idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)
rescaledData = idfModel.transform(featurizedData)

rescaledData.select("id", "features").show()

+----+--------------------+
|  id|            features|
+----+--------------------+
|  26|(5,[0,1,2,3,4],[0...|
|  29|(5,[0,1,2,4],[0.3...|
| 474|(5,[0,1,2,3],[0.7...|
| 964|(5,[0,1,4],[1.419...|
|1677|(5,[0,1,2,3,4],[0...|
|1697|(5,[0,1,2,4],[0.3...|
|1806|(5,[1,2,3,4],[0.3...|
|1950|(5,[1,3,4],[0.750...|
|2040|(5,[0,1],[1.06440...|
|2214|(5,[3,4],[0.45814...|
|2250|(5,[0,1,4],[0.354...|
|2453|(5,[0,2,3],[0.354...|
|2509|(5,[1,3,4],[1.126...|
|2529|(5,[0,2,4],[2.128...|
|2927|(5,[0,1,3],[0.354...|
|3091|(5,[0,1,2,3,4],[0...|
|3506|(5,[0,1,2,3,4],[1...|
|3764|(5,[0,4],[0.70960...|
|4590|(5,[0,1,4],[0.709...|
|4823|(5,[0,3,4],[0.354...|
+----+--------------------+
only showing top 20 rows



In [0]:
rescaledData.select("id", "features").show(n=5, truncate=False, vertical=True)

-RECORD 0------------------------------------------------------------------------------------------------------------------------
 id       | 26                                                                                                                   
 features | (5,[0,1,2,3,4],[0.35480052914438215,0.37534179192543426,0.48835740044961995,1.3744254057774885,0.40377797676786836]) 
-RECORD 1------------------------------------------------------------------------------------------------------------------------
 id       | 29                                                                                                                   
 features | (5,[0,1,2,4],[0.35480052914438215,1.1260253757763028,0.48835740044961995,1.211333930303605])                         
-RECORD 2------------------------------------------------------------------------------------------------------------------------
 id       | 474                                                                           

In [0]:
trainSDF,validationSDF,TestSDF=rescaledData.select("id","Tokenized_tags").randomSplit([0.75, 0.2, 0.05])

In [47]:
trainSDF.show(5)

+----+--------------------+
|  id|      Tokenized_tags|
+----+--------------------+
|  26|[grey hair, sophd...|
|  29|[moving vlog, ike...|
| 474|[PC, IGN, PS4, Do...|
| 964|[Google, YouTube,...|
|1677|[Project Cars 2 G...|
+----+--------------------+
only showing top 5 rows



In [48]:
validationSDF.show(5)

+----+--------------------+
|  id|      Tokenized_tags|
+----+--------------------+
|1697|[Shakira ft. Nick...|
|2040|[Noah Cyrus, Noah...|
|2250|[The Rock, The Ro...|
|3764|[Football League,...|
|5385|[my little pony, ...|
+----+--------------------+
only showing top 5 rows



In [49]:
TestSDF.count()

814

In [51]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import HashingTF, MinHashLSH
import pyspark.sql.functions as f

model = Pipeline(stages=[
        HashingTF(inputCol="Tokenized_tags", outputCol="vectors"),
        MinHashLSH(inputCol="vectors", outputCol="lsh", numHashTables=10)
    ]).fit(TestSDF)

db_hashed = model.transform(TestSDF)

db_matches = model.stages[-1].approxSimilarityJoin(db_hashed, db_hashed, 0.9)

#show all matches (including duplicates)
db_matches.select(f.col('datasetA.id').alias('video_id_A'),
                 f.col('datasetB.id').alias('video_id_B'),
                 f.col('distCol')).show()


+----------+----------+------------------+
|video_id_A|video_id_B|           distCol|
+----------+----------+------------------+
|      3665|      7294|0.8571428571428572|
|      3846|8589937514|               0.0|
|8589939305|      1664|               0.0|
|      1291|      2610|               0.0|
|8589937514|      1291|               0.0|
|8589939807|      4508|               0.0|
|8589938440|8589935878|               0.0|
|8589936990|8589935855|               0.0|
|      6125|      6125|               0.0|
|      6507|      6507|               0.0|
|8589938902|       774|0.6666666666666667|
|      6392|      6392|               0.0|
|8589936410|8589936410|               0.0|
|       877|8589935666|               0.0|
|      6912|      6912|               0.0|
|8589936953|8589936953|               0.0|
|8589937057|8589939099|               0.0|
|      3846|      5361|               0.0|
|8589941701|      3008|               0.0|
|8589940332|8589939305|               0.0|
+----------

In [53]:
from pyspark.sql.functions import col
#show non-duplicate matches
ans=db_matches.select(f.col('datasetA.id').alias('video_id_A'),
                 f.col('datasetB.id').alias('video_id_B'),
                 f.col('distCol')).filter('video_id_A < video_id_B')

ans=ans.sort(col("distCol").desc())
ans.show()

+----------+----------+------------------+
|video_id_A|video_id_B|           distCol|
+----------+----------+------------------+
|       774|      1787|0.8974358974358975|
|       251|      1498|0.8970588235294118|
|      1498|8589935110|0.8970588235294118|
|      6981|8589940691| 0.896551724137931|
|       566|8589937747|0.8958333333333334|
|      6185|8589936518|0.8958333333333334|
|8589937619|8589938253|0.8958333333333334|
|8589937393|8589938824|0.8947368421052632|
|8589937865|8589938824|0.8947368421052632|
|      3901|8589935355|0.8947368421052632|
|      2469|8589942585|0.8947368421052632|
|8589937609|8589938824|0.8947368421052632|
|8589936622|8589936639|0.8947368421052632|
|      7029|8589935339|0.8947368421052632|
|8589935742|8589941992|0.8947368421052632|
|      2910|      5764|0.8947368421052632|
|8589936622|8589938973|0.8947368421052632|
|8589935355|8589937619|0.8947368421052632|
|8589936622|8589938060|0.8947368421052632|
|       832|      4847|0.8928571428571429|
+----------

In [0]:
set1,set2,set3=ans.select("video_id_A","video_id_B","distCol").randomSplit([0.33, 0.33, 0.34])

In [57]:
set1.show()

+----------+----------+------------------+
|video_id_A|video_id_B|           distCol|
+----------+----------+------------------+
|       918|      3008|               0.0|
|8589937057|8589939099|               0.0|
|       918|      2610|               0.0|
|      5228|8589940204|0.6382978723404256|
|      6799|8589936922|               0.0|
|      3407|8589934640|0.6060606060606061|
|      6478|      7351|               0.0|
|      6535|8589941701|               0.0|
|      6535|8589935247|               0.0|
|      1758|8589941901|               0.0|
|      1918|8589936990|               0.0|
|      2610|8589935878|               0.0|
|      4508|8589939371|               0.0|
|8589935344|8589935576|               0.0|
|8589935855|8589940335|               0.0|
|8589937057|8589941701|               0.0|
|8589937758|8589941901|               0.0|
|8589939305|8589940335|               0.0|
|      1301|      4508|               0.0|
|      1758|8589938440|               0.0|
+----------

In [59]:
set2.show()

+----------+----------+------------------+
|video_id_A|video_id_B|           distCol|
+----------+----------+------------------+
|      3846|      5361|               0.0|
|      3946|8589936288|0.5714285714285714|
|      4000|      6535|               0.0|
|      4000|8589939300|               0.0|
|      4000|8589940758|               0.0|
|      5632|8589935247|               0.0|
|8589934666|8589941901|               0.0|
|8589935802|8589936298|               0.0|
|8589937283|8589939134|0.7692307692307692|
|      6535|      6799|               0.0|
|8589935878|8589938618|               0.0|
|8589935878|8589940332|               0.0|
|8589936990|8589940690|               0.0|
|8589939300|8589940690|               0.0|
|      2166|8589937514|               0.0|
|      5361|8589938618|               0.0|
|8589935855|8589940332|               0.0|
|       758|8589940324|0.7115384615384616|
|       918|8589940690|               0.0|
|      7351|8589940332|               0.0|
+----------

In [56]:
set3.show()

+----------+----------+------------------+
|video_id_A|video_id_B|           distCol|
+----------+----------+------------------+
|      1291|      2610|               0.0|
|      3665|      7294|0.8571428571428572|
|      3846|8589937514|               0.0|
|       877|8589935666|               0.0|
|      2610|      7456|               0.0|
|      1301|8589940335|               0.0|
|      5632|8589939300|               0.0|
|8589936990|8589941901|               0.0|
|      2610|8589939305|               0.0|
|8589936311|8589937495|0.8333333333333334|
|      3846|8589935247|               0.0|
|      6971|8589935855|               0.0|
|8589937283|8589937696|0.7727272727272727|
|      4847|      7864|0.8857142857142857|
|8589938618|8589939807|               0.0|
|      1758|8589937758|               0.0|
|      3407|8589935703|0.5806451612903225|
|      3008|      6478|               0.0|
|      4804|      6608| 0.303030303030303|
|8589935371|8589940509|0.8867924528301887|
+----------

In [0]:
# from pyspark.ml.feature import HashingTF, IDF
# hashingTF = HashingTF(inputCol="Tokenized_Tags", outputCol="tf")
# tf = hashingTF.transform(TestSDF.select("id","Tokenized_Tags"))

# idf = IDF(inputCol="tf", outputCol="feature").fit(tf)
# tfidf = idf.transform(tf)

In [0]:
# #   tf.cache()
# #   idf = IDF().fit(tf)
# #   tfidf = idf.transform(tf)

# from pyspark.ml.feature import Normalizer
# normalizer = Normalizer(inputCol="feature", outputCol="norm")
# data = normalizer.transform(tfidf)

In [0]:
# data.show(5)

+----------+--------------------+--------------------+--------------------+--------------------+
|        id|      Tokenized_Tags|                  tf|             feature|                norm|
+----------+--------------------+--------------------+--------------------+--------------------+
|      3091|[affordable makeu...|(262144,[5262,199...|(262144,[5262,199...|(262144,[5262,199...|
|      4894|[Linkin Park, One...|(262144,[21656,84...|(262144,[21656,84...|(262144,[21656,84...|
|      7279|[carlibel55, carl...|(262144,[10116,37...|(262144,[10116,37...|(262144,[10116,37...|
|8589938024|[Full Frontal wit...|(262144,[27323,66...|(262144,[27323,66...|(262144,[27323,66...|
|8589942265|        [nerdwriter]|(262144,[109605],...|(262144,[109605],...|(262144,[109605],...|
+----------+--------------------+--------------------+--------------------+--------------------+
only showing top 5 rows



In [0]:

# from pyspark.sql.types import DoubleType

In [0]:
# import pyspark.sql.functions as psf
# from pyspark.sql.functions import col
# dot_udf = psf.udf(lambda x,y: float(x.dot(y)), DoubleType())
# result=data.alias("i").join(data.alias("j"), psf.col("i.ID") < psf.col("j.ID"))
# result=result.select(
#         psf.col("i.ID").alias("i"), 
#         psf.col("j.ID").alias("j"), 
#         dot_udf("i.norm", "j.norm").alias("dot"))
    
# # result=result.sort("i", "j")

# result=result.sort(col("dot").desc())
# result.show()

+----------+----------+------------------+
|         i|         j|               dot|
+----------+----------+------------------+
|      2727|8589936472|1.0000000000000007|
|      5060|      5738|1.0000000000000007|
|      7260|      7741|1.0000000000000004|
|      1652|      2366|1.0000000000000004|
|       610|      1262|1.0000000000000002|
|      2759|8589936482|1.0000000000000002|
|      6152|8589939207|1.0000000000000002|
|      4116|8589938314|1.0000000000000002|
|8589938210|8589938672|1.0000000000000002|
|      4135|      4383|1.0000000000000002|
|      5459|8589939207|1.0000000000000002|
|      4417|      5312|1.0000000000000002|
|      7402|8589942486|1.0000000000000002|
|      6015|      7097|1.0000000000000002|
|8589940276|8589940529|1.0000000000000002|
|8589936018|8589936775|1.0000000000000002|
|      5750|      6418|1.0000000000000002|
|      5459|      6152|1.0000000000000002|
|      5213|      6172|1.0000000000000002|
|      7402|8589941791|1.0000000000000002|
+----------

In [0]:

# result.sort(col("dot").desc())

In [0]:
# content based recommender system
# steps: 
# 1) lower case whole dataset
# 2) remove stopwords
# 3) create bag of words vectors in binary format 1/0
# 4) calc cosine sim for all articles and find top 3 closest articles
# 5) sort article ids with descen simi and removing same article id
# 6) created recommendation column with top 3 article id's